# Problem 1
## a. Using variable elimination, calculate the probability that a student who did well on the exam understands the material.P(+u|+e). Show your work.

P(+e) = 

# Problem 2

In [25]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
data = pd.read_csv("Life Expectancy Data.csv")
data.describe()

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


In [70]:
# To find correlation of each variable perform linear regression in relation to each variable
print(data.values.shape)
filtered = data.values[:,:]
print("filtered")
filtered=filtered[:,3:]
print(filtered)
for col in filtered.T:
    col = col.astype(float)
    col = np.array(col)
    print(col.astype(float))
    print(repr(col))
    col = col[~np.isnan(col)]
    print(np.isnan(col))
    reg = LinearRegression().fit(col.reshape(-1,1), data.values[2:])
    print(reg.score())
    


(2938, 22)
filtered
[[65.0 263.0 62 ... 17.3 0.479 10.1]
 [59.9 271.0 64 ... 17.5 0.47600000000000003 10.0]
 [59.9 268.0 66 ... 17.7 0.47 9.9]
 ...
 [44.8 73.0 25 ... 1.3 0.42700000000000005 10.0]
 [45.3 686.0 25 ... 1.7 0.42700000000000005 9.8]
 [46.0 665.0 24 ... 11.2 0.434 9.8]]
[65.  59.9 59.9 ... 44.8 45.3 46. ]
array([65. , 59.9, 59.9, ..., 44.8, 45.3, 46. ])
[False False False ... False False False]


ValueError: Input contains NaN

In [ ]:
# To make model perform regresion on data set
